# Data Preparation; Clean, Preprocess, Normalize, Standardize

## File: Bewegungen.csv

In [1]:
import pandas as pd

# List of columns to keep (ran into memory issues...)
columns_to_keep = [
    "STATUS", "SRC_ITEM", "SRC_LOT", "SRC_QACODE",
    "DST_QACODE", "SRC_WA", "SRC_X", "SRC_Y", "SRC_Z", "OUTNUM", "CONQTY",
    "LISNUM", "SUMLIS", "TRNDAT", "USERID", # , "TRNNUM", "WORNUM", "DST_LOT", "DST_WA", "LOADDAT", "CRTDAT"
]

dtypes = {
#    "WORNUM": "int32",
    "STATUS": "int32",
#    "MOVTYP": "category",
#    "MOVKEY": "category",
    "SRC_ITEM": "category",
    "SRC_LOT": "string",
    "SRC_QACODE": "category",
    #"DST_LOT": "string",
    "DST_QACODE": "category",
    "SRC_WA": "category",
    "SRC_X": "category",
    "SRC_Y": "category",
    "SRC_Z": "category",
#    "DST_WA": "category",
    "CONQTY": "int32",
    "OUTNUM": "int32", # KEY
#    "RELNUM": "int32",
    "LISNUM": "category",
    "SUMLIS": "category",
#    "TRNNUM": "int32",
    "USERID": "category",
}

# Load the CSV with optimized settings, and only load necessary cols
df_bewegungen = pd.read_csv(
    '../Data/bewegungen.csv',
    usecols=columns_to_keep,  # Only load the specified columns
    dtype=dtypes, # use optimized, manually set data types
    parse_dates=["TRNDAT"], # have pandas parse the date
    low_memory=False # I set this initially to False to have more accurate dtype assignments, can be set to low_memory to improve performance
)

####
# Data Cleaning
####

# Temporarily convert 'SRC_LOT' to string type for cleaning - (Column "Artikelcharge", this LOT usually is a 3 digit int. An Article can have multiple LOTs. I simplify by removing leading zeros and clean up the column from wrong manual usererrors.
df_bewegungen['SRC_LOT'] = df_bewegungen['SRC_LOT'].astype(str)  # Convert Categorical to string
# Clean and validate the SRC_LOT column
def clean_lot(value):
    value = value.lstrip('0') if value != '000' else value  # Remove leading zeros unless '000'
    if value.isdigit() and 1 <= len(value) <= 3:           # Validate length (1 to 3 digits)
        return value.zfill(3)                              # Pad with leading zeros to ensure 3 digits
    return None                                            # Remove invalid entries

df_bewegungen['SRC_LOT'] = df_bewegungen['SRC_LOT'].apply(clean_lot)
# Convert back to Categorical
df_bewegungen['SRC_LOT'] = pd.Categorical(df_bewegungen['SRC_LOT'])

# Filter STATUS column (10= Offen, 50= Bestätigt, 95= Storniert -- for my purposes I'm only interested in "Bestätigt" rows.
# Filter the DataFrame to keep only rows with status == 50 (abgeschlossen)
df_bewegungen = df_bewegungen[df_bewegungen['STATUS'] == 50]

# Filter the DataFrame in place to include only rows where 'SRC_WA' contains "WA", excluding all 'WE', 'WA', 'AU' and 'UM' areas
df_bewegungen = df_bewegungen[df_bewegungen['SRC_WA'] == 'EG']

# Remove rows where USER == 'LXONE'
df_bewegungen = df_bewegungen[df_bewegungen['USERID'] != 'LXONE']

# Only keep rows with QACODE == 'H'
df_bewegungen = df_bewegungen[df_bewegungen['SRC_QACODE'] == 'H']

# Remove all additional Crossdocking movements, easiest to pinpoint via SRC_LOT = 000
df_bewegungen = df_bewegungen[df_bewegungen['SRC_LOT'] != '000']


# Combine SRC_X, SRC_Y, and SRC_Z into a new column STOR_LOC
df_bewegungen['STOR_LOC'] = df_bewegungen['SRC_X'].astype(str) + \
                            df_bewegungen['SRC_Y'].astype(str) + \
                            df_bewegungen['SRC_Z'].astype(str)

# Change datatypes as needed
df_bewegungen = df_bewegungen.astype({
#    "SRC_LOT": "category",
#    "DST_LOT": "category",
    "STOR_LOC": "category",
})

# Drop the original SRC_X, SRC_Y, and SRC_Z columns
df_bewegungen.drop(['SRC_X', 'SRC_Y', 'SRC_Z'], axis=1, inplace=True)

# Drop rows with missing values from the DataFrame - dropna (by default, without parameters) removes entire rows which have a NaN or null value
df_bewegungen.dropna(inplace=True)

# Drop both 'SRC_WA' and 'DST_WA' columns from the DataFrame now that we filtered the data
df_bewegungen.drop(columns=['SRC_LOT', 'STATUS', 'SRC_WA', 'SRC_QACODE', 'DST_QACODE'], inplace=True)

# Remove unused categories (i.e. Artifacts) across all categorical columns (after all cleaning steps there were pandas retained all defined categories, even though they no longer existed in the data. Not sure if this would've led to problems down the road during embedding/labeling, so I removed them to avoid eventual issues)
for col in df_bewegungen.select_dtypes(include='category').columns:
    df_bewegungen[col] = df_bewegungen[col].cat.remove_unused_categories()

# Verify the changes
print(f"Updated DataFrame shape: {df_bewegungen.shape}")
# :::

Updated DataFrame shape: (17914909, 8)


In [7]:
df_bewegungen.info()
df_bewegungen.head()
print(f"DataFrame shape: {df_bewegungen.shape}")

<class 'pandas.core.frame.DataFrame'>
Index: 17914909 entries, 0 to 20094144
Data columns (total 8 columns):
 #   Column    Dtype         
---  ------    -----         
 0   SRC_ITEM  category      
 1   CONQTY    int32         
 2   OUTNUM    int32         
 3   LISNUM    category      
 4   SUMLIS    category      
 5   TRNDAT    datetime64[ns]
 6   USERID    category      
 7   STOR_LOC  category      
dtypes: category(5), datetime64[ns](1), int32(2)
memory usage: 818.0 MB


TypeError: 'tuple' object is not callable

As I cannot provide the full file:
OUTPUT: DataFrame shape: (17914909, 8) -- 818.0 MB

Filter the Data to only contain Transactiondates > 2024-11-20 -- base is for proof of concept, otherwise - due to extreme cardinality - the computation times are too long.

In [2]:
import pandas as pd
# Define the fixed date for filtering 2024-11-20
start_date = pd.Timestamp('2024-11-20')

# Filter the DataFrame to include only rows with TRNDAT newer than 01.01.2023
df_bewegungen_reduc = df_bewegungen[df_bewegungen['TRNDAT'] > start_date]

# Verify the result
print(f"Shape of DataFrame for data newer than 2024-11-20: {df_bewegungen_reduc.shape}")

Shape of DataFrame for data newer than 2024-11-20: (61014, 8)


As I cannot provide the full file:
OUTPUT: Shape of DataFrame for data newer than 2024-11-20: (61014, 8) -- 102.9 MB

In [8]:
df_bewegungen_reduc.info()
df_bewegungen_reduc.head()
print(f"DataFrame shape: {df_bewegungen_reduc.shape}")

<class 'pandas.core.frame.DataFrame'>
Index: 61014 entries, 19728799 to 20074575
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   SRC_ITEM  61014 non-null  category      
 1   CONQTY    61014 non-null  int32         
 2   OUTNUM    61014 non-null  int32         
 3   LISNUM    61014 non-null  category      
 4   SUMLIS    61014 non-null  category      
 5   TRNDAT    61014 non-null  datetime64[ns]
 6   USERID    61014 non-null  category      
 7   STOR_LOC  61014 non-null  category      
dtypes: category(5), datetime64[ns](1), int32(2)
memory usage: 102.9 MB
Updated DataFrame shape: (61014, 8)


## File: wa_kopf.csv

In [5]:
# List of columns to keep (ran into memory issues...)
columns_to_keep = [
    "OUTNUM", "DOCNUM", "STATUS", "PICCOD", "CUSNUM",
    "SHPTYP", "TOUR", # "TRNDAT" "CRTDAT", "ORDDAT", "DLVDAT", not needed rn
]

dtypes = {
    "OUTNUM": "int32", # KEY
    "DOCNUM": "category", # Warenausgangsnummer
    # "ORDNUM": "category", # dont think i need this, achieves the same as DOCNUM
    "STATUS": "int32",
    "PICCOD": "category",
    "CUSNUM": "category", # kunde
    "SHPTYP": "category", # versandart
    "TOUR": "category",
}

df_wa_kopf = pd.read_csv(
    '../Data/wa_kopf.csv',
    usecols=columns_to_keep,  # Only load the specified columns
    dtype=dtypes,             # Use optimized data types
    #usecols=columns_to_use,  # Only load required columns
    # parse_dates=["TRNDAT"], # "ORDDAT", "DLVDAT", "CRTDAT",  not needed rn
    low_memory=False
)
# Filter the DataFrame to keep only rows with status == 90 (abgeschlossene)
df_wa_kopf = df_wa_kopf[df_wa_kopf['STATUS'] == 90]

# Drop rows with missing values from the DataFrame - some early data and tests lead to PICCOD, SHPTYP and TOUR being empty (~86 rows)
df_wa_kopf.dropna(inplace=True) # inplace=True modifies DataFrame directly without having to create a new one

# Drop 'STATUS' column from the DataFrame now that we filtered the data
df_wa_kopf.drop(columns=['STATUS'], inplace=True)

OUTPUT: DateFrame Shape: (19314904, 6) -- 581.1 MB

In [9]:
df_wa_kopf.info()
df_wa_kopf.head()
print(f"DataFrame shape: {df_wa_kopf.shape}")

<class 'pandas.core.frame.DataFrame'>
Index: 19314904 entries, 0 to 19377912
Data columns (total 6 columns):
 #   Column  Dtype   
---  ------  -----   
 0   OUTNUM  int32   
 1   DOCNUM  category
 2   PICCOD  category
 3   CUSNUM  category
 4   SHPTYP  category
 5   TOUR    category
dtypes: category(5), int32(1)
memory usage: 581.1 MB
Updated DataFrame shape: (19314904, 6)


## File: wa_positionen

! File currently not in use due to memory constraints and no use-case that can use the SHPQTY, RELQTY and RELQTY values yet

In [19]:
# List of columns to keep (ran into memory issues...)
columns_to_keep = [
    "OUTNUM", "STATUS", # "LOT",
    "ORDQTY", "CONQTY", "USERID", "TRNNUM" #  "TRNDAT", "CRTDAT", not needed rn
]

dtypes = {
    "OUTNUM": "int32",
    "STATUS": "int32",
#    "LOT": "string", # 3 character string, e.G: 001, 002, 006, 012 etc.
    "ORDQTY": "int32",
#    "RELQTY": "int32" # freigegebene Menge
#    "FNDQTY": "int32" # reservierte Menge
#    "SHPQTY": "int32" # versendete Menge
    "CONQTY": "float32", # setting to float for initial load, need to clean the dataframe and change it to int later - otherwise runs into error because of existing float values
    "USERID": "category",
    "TRNNUM": "int32"
}

# Load the CSV with optimized settings, and only load necessary cols
df_wa_positionen = pd.read_csv(
    '../Data/wa_positionen.csv',
    usecols=columns_to_keep,  # Only load the specified columns
    dtype=dtypes,
    #usecols=columns_to_use,  # Only load required columns
    #dtype=dtypes,            # Use optimized data types
    # parse_dates=["TRNDAT","CRTDAT"] # , not needed rn
    low_memory=False
)

# Drop rows with missing values from the DataFrame
df_wa_positionen.dropna(inplace=True)

# Drop rows where CONQTY has a fractional part (i.e., a value with anything after the decimal point)
df_wa_positionen = df_wa_positionen[df_wa_positionen["CONQTY"] % 1 == 0]

# Convert CONQTY column to integer type to reflect that it no longer has fractions
df_wa_positionen["CONQTY"] = df_wa_positionen["CONQTY"].astype(int)

Dataframe shape: (19375223, 7) 741.3 MB

In [20]:
df_wa_positionen.info()
df_wa_positionen.head()
print(f"DataFrame shape: {df_wa_positionen.shape}")

<class 'pandas.core.frame.DataFrame'>
Index: 19375223 entries, 0 to 19375223
Data columns (total 7 columns):
 #   Column  Dtype   
---  ------  -----   
 0   OUTNUM  int32   
 1   STATUS  int32   
 2   ITEM    category
 3   ORDQTY  int32   
 4   CONQTY  int64   
 5   USERID  category
 6   TRNNUM  int32   
dtypes: category(2), int32(4), int64(1)
memory usage: 741.3 MB
Updated DataFrame shape: (19375223, 7)


# Join Tables

For demonstration im only joining the reduced dataframe "df_bewegungen_reduc" onto the dataframe df_wa_kopf (without df_wa_positionen) to get a small joined dataframe that I can use as basis for proof of concept. (Considering using the entire Dataset would never be able to compute in a timely manner (years :)) on my Machine with no GPU-Acceleration.)

In [10]:
df_full = (
    pd.merge(df_bewegungen_reduc, df_wa_kopf, on='OUTNUM', how='inner')
)

df_full.drop(columns=['OUTNUM'], inplace=True)

shape: (60758, 12) - 278.7 MB

In [11]:
(df_full.info())
df_full.head()
print(f"DataFrame shape: {df_full.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60758 entries, 0 to 60757
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   SRC_ITEM  60758 non-null  category      
 1   CONQTY    60758 non-null  int32         
 2   LISNUM    60758 non-null  category      
 3   SUMLIS    60758 non-null  category      
 4   TRNDAT    60758 non-null  datetime64[ns]
 5   USERID    60758 non-null  category      
 6   STOR_LOC  60758 non-null  category      
 7   DOCNUM    60758 non-null  category      
 8   PICCOD    60758 non-null  category      
 9   CUSNUM    60758 non-null  category      
 10  SHPTYP    60758 non-null  category      
 11  TOUR      60758 non-null  category      
dtypes: category(10), datetime64[ns](1), int32(1)
memory usage: 278.7 MB
Updated DataFrame shape: (60758, 12)


This Dataframe "df_full" shall be the basis for a "Market Basket Analysis (MBA)" -> I'm writing the DF into a .parquet file, which I will provide in my linked Onedrive Folder. (28 MB)

# Market Basket Analysis

In [14]:
import pandas as pd
import pyarrow

# Extract the relevant columns into a new DataFrame
df_mba = df_full[['SRC_ITEM', 'DOCNUM']]

# Save the DataFrame to a Parquet file
df_mba.to_parquet('df_mba.parquet', index=False)

print("Data extracted and saved to df_mba.parquet successfully.")

Data extracted and saved to df_mba.parquet successfully.


The following code find frequent itemsets and generates association rules from raw transactional data.

-Loads and prunes data (rare items, empty transactions) for efficiency
-one-hot encodes transactions
-uses FP Growth algorithm to find frequent itemsets.
-then uses frequent itemsets to generate association rules

In [18]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the parquet file
df = pd.read_parquet("df_mba.parquet", columns=["SRC_ITEM", "DOCNUM"])

# Optional Pruning: Keep items with minimum support -- considering computational limitations I had to play around with these values to receive an Output
min_item_support = 20  # Adjust threshold based on data size
item_counts = df["SRC_ITEM"].value_counts()
frequent_items = item_counts[item_counts >= min_item_support].index
df = df[df["SRC_ITEM"].isin(frequent_items)]

# Remove unused categories
df["SRC_ITEM"] = df["SRC_ITEM"].astype('category')
df["SRC_ITEM"] = df["SRC_ITEM"].cat.remove_unused_categories()

# Group transactions (explicitly set observed=False to silence warnings) - produces a list of items for each DOCNUM
transactions = df.groupby("DOCNUM", observed=False)["SRC_ITEM"].apply(list)
transactions = transactions[transactions.apply(len) > 0]  # Remove empty transactions
print(f"Number of transactions after filtering empty ones: {len(transactions)}")
print(transactions.head()) # debugging purposes to see how transactions are stored

# Encode transactions using `sparse=True` - transforms data into a one-hot encoded (sparse) matrix format for memory efficiency
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions, sparse=True)

# Convert the sparse matrix to a sparse DataFrame with explicit SparseDtype
df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_).astype(pd.SparseDtype("bool", fill_value=False))

print(f"Encoded DataFrame shape: {df_encoded.shape}")
# Apply FP-Growth
min_support = 0.001  # Lower support threshold - i.e. with 100k entries, a support threshold of 0.01 means 1% of all transactions
frequent_itemsets = fpgrowth(df_encoded, min_support=min_support, use_colnames=True)

# Filter itemsets with more than one item
#frequent_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) > 1)]

# Check if itemsets are found
if frequent_itemsets.empty:
    print("No frequent itemsets found. reduce `min_support` and adjust pruning.")
else:
    # Add the `num_itemsets` column required for association_rules
    # Calculate number of transactions
    num_itemsets = len(transactions)  # Total number of transactions

    print("Frequent Itemsets:")
    print(frequent_itemsets)

    # Generate association rules - num_itemsets should be size of transactions to do correct statistical calculations
    # lift = confidence/expected confidence if items independent, increasing min_threshold will lead to fewer rules -> strongest correlations only
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0, num_itemsets=num_itemsets)
    print("\nAssociation Rules:")
    print(rules)


Number of transactions after filtering empty ones: 5008
DOCNUM
02713239e                                            [2311534]
02713241e    [46446861, 46446861, 46446861, 46446861, 46446...
02713314e                                           [46365590]
02714106e                                           [47309357]
16579459                                  [46382948, 46382948]
Name: SRC_ITEM, dtype: object
Encoded DataFrame shape: (5008, 176)


C:\Users\Manu\AppData\Local\Temp\ipykernel_25760\2268555314.py:29: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_).astype(pd.SparseDtype("bool", fill_value=False))


Frequent Itemsets:
      support                        itemsets
0    0.003994                       (2311534)
1    0.002196                      (46446861)
2    0.004593                      (46365590)
3    0.039337                      (47309357)
4    0.026158                      (46382948)
..        ...                             ...
819  0.001398  (46347654, 46349329, 45508974)
820  0.001797            (46365742, 46364898)
821  0.002596            (46349329, 46365742)
822  0.001997            (47309331, 46365742)
823  0.001198  (47309331, 46349329, 46365742)

[824 rows x 2 columns]

Association Rules:
               antecedents           consequents  antecedent support  \
0               (43112467)            (46048383)            0.013379   
1               (46048383)            (43112467)            0.013778   
2               (46357429)            (46358239)            0.006190   
3               (46358239)            (46357429)            0.011182   
4               (46357429